**WEB SCRAPING AEMET**

El objetivo de este código es recolectar datos meteorológicos del servicio de AEMET para el municipio de Vilassar de Mar desde marzo de 2021 hasta 2025.

Estos datos incluyen información sobre temperaturas máximas, mínimas, medias y precipitaciones por hora de cada dia. Con este código se obtiene mediante técnicas de web scraping esta información y se calcula la temperatura media diaria y la precipitación diaria.

El motivo de esta recolección es incorporar la variable climática al estudio de predicción de ocupación de un B&B, ya que las condiciones meteorológicas pueden influir en la demanda turística y, por tanto, en la ocupación del establecimiento.

Este módulo forma parte de un proyecto más amplio orientado a aplicar inteligencia artificial en la predicción de ocupación hotelera.

Debido a que el servidor tiende a bloquear el acceso a la URL cuando detecta actividad de scraping, se decidió realizar la extracción de datos en bloques de 60 días, utilizando además tiempos de espera aleatorios tanto entre días como entre bloques para reducir la probabilidad de detección.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import time, random
import os

# --- Función para extraer datos de un día ---
def extraer_datos_dia(url, fecha_str):
    user_agents = [
    # Navegadores modernos en Windows
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; rv:109.0) Gecko/20100101 Firefox/120.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/120.0.0.0 Safari/537.36",

    # macOS
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_6_9) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",

    # Linux
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) Gecko/20100101 Firefox/117.0",

    # Android
    "Mozilla/5.0 (Linux; Android 13; Pixel 6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.144 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 11; SM-G991B) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36",

    # iOS
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (iPad; CPU OS 16_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Mobile/15E148 Safari/604.1"
]

    try:
        headers = {'User-Agent': random.choice(user_agents)}
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        tabla = soup.select_one('div.div_table_scroll table')
        if not tabla:
            return None

        # Identificar cabeceras
        cabecera = tabla.find("tr")
        ths = [th.text.strip().upper() for th in cabecera.find_all("th")]

        # Buscar índices según encabezados
        idx_temp = next((i for i, texto in enumerate(ths) if "TEMP" in texto), None)
        idx_prec = next((i for i, texto in enumerate(ths) if "PRECIPITACIÓN" in texto or "PREC" in texto), None)

        if idx_temp is None or idx_prec is None:
            print(f"⚠️ No se encontraron columnas adecuadas en {fecha_str}")
            return None

        # Filas de datos
        filas = tabla.find_all("tr")[1:]
        temps, precs = [], []

        for fila in filas:
            columnas = fila.find_all("td")
            if len(columnas) > max(idx_temp, idx_prec):
                try:
                    temp = float(columnas[idx_temp].text.replace(",", "."))
                    prec = float(columnas[idx_prec].text.replace(",", "."))
                    temps.append(temp)
                    precs.append(prec)
                except ValueError:
                    continue

        if temps:
            return {
                "fecha": fecha_str,
                "temp_media": round(sum(temps) / len(temps), 2),
                "precipitacion": round(sum(precs), 2)
            }
        else:
            return None

    except Exception as e:
        print(f"❌ Error en {fecha_str}: {e}")
        return None

# --- Parámetros de scraping ---
base_url = "https://x-y.es/aemet/est-0244X-vilassar-de-dalt?fecha={}"
fecha_inicio = datetime(2021, 3, 31)
fecha_fin = datetime(2025, 4, 19)
bloque_dias = 60

# --- Crear bloques de fechas ---
rango_total = pd.date_range(start=fecha_inicio, end=fecha_fin)
bloques = [rango_total[i:i + bloque_dias] for i in range(0, len(rango_total), bloque_dias)]

# --- Proceso por bloques ---
csv_files = []

for i, bloque in enumerate(bloques):
    print(f"\n🔄 Procesando bloque {i+1}/{len(bloques)}: {bloque[0].date()} → {bloque[-1].date()}")
    datos_bloque = []

    for fecha in tqdm(bloque):
        fecha_str = fecha.strftime("%Y-%m-%d")
        url = base_url.format(fecha_str)
        resultado = extraer_datos_dia(url, fecha_str)
        if resultado:
            datos_bloque.append(resultado)

        time.sleep(random.uniform(1.1, 2.5))


    if datos_bloque:
        df_bloque = pd.DataFrame(datos_bloque)
        archivo_csv = f"datos_bloque_{i}.csv"
        df_bloque.to_csv(archivo_csv, index=False)
        csv_files.append(archivo_csv)
        print(f"✅ Guardado: {archivo_csv}")

    # Pausa larga entre bloques para evitar bloqueo
    if i < len(bloques) - 1:
        pausa = random.uniform(300, 600)
        print(f"⏸️ Pausa de {int(pausa)} segundos antes del siguiente bloque...")
        time.sleep(pausa)

# --- Unir todos los CSVs ---
df_final = pd.concat([pd.read_csv(f) for f in csv_files])
df_final['fecha'] = pd.to_datetime(df_final['fecha'])
df_final = df_final.sort_values('fecha').reset_index(drop=True)

# --- Guardar CSV final ---
df_final.to_csv("clima_diario_vilassar_2021_2025.csv", index=False)
print("\n📦 Archivo final guardado como 'clima_diario_vilassar_2021_2025.csv'")

# Mostrar resumen
df_final.head()



🔄 Procesando bloque 1/25: 2021-03-31 → 2021-05-29


  0%|          | 0/60 [00:00<?, ?it/s]

⏸️ Pausa de 495 segundos antes del siguiente bloque...

🔄 Procesando bloque 2/25: 2021-05-30 → 2021-07-28


  0%|          | 0/60 [00:00<?, ?it/s]

⏸️ Pausa de 517 segundos antes del siguiente bloque...

🔄 Procesando bloque 3/25: 2021-07-29 → 2021-09-26


...

⏸️ Pausa de 476 segundos antes del siguiente bloque...

🔄 Procesando bloque 24/25: 2025-01-09 → 2025-03-09


  0%|          | 0/60 [00:00<?, ?it/s]

⏸️ Pausa de 445 segundos antes del siguiente bloque...

🔄 Procesando bloque 25/25: 2025-03-10 → 2025-04-19


  0%|          | 0/41 [00:00<?, ?it/s]

ValueError: No objects to concatenate

A pesar de implementar medidas para evitar el bloqueo del servidor, no fue posible recuperar todos los datos de forma continua. Por ello, se optó por desarrollar un script que ejecuta la recolección únicamente para los primeros 60 días con datos faltantes en el dataset. Al finalizar cada iteración, se desconecta manualmente el entorno de ejecución, lo cual permite cambiar la dirección IP y así eludir temporalmente el bloqueo del servidor.

In [ ]:
# ✅ 1. Importar librerías
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time

# ✅ 2. Cargar CSV existente
nombre_csv_existente = "clima_vilassar_completo_actualizado.csv"  # Cambia si tu archivo tiene otro nombre
df_existente = pd.read_csv(nombre_csv_existente, parse_dates=["fecha"])
print(f"📄 CSV original cargado con {len(df_existente)} registros")

# ✅ 3. Detectar fechas faltantes
fecha_inicio = datetime(2021, 3, 31)
fecha_fin = datetime(2025, 4, 19)
fechas_completas = pd.date_range(start=fecha_inicio, end=fecha_fin)

fechas_en_csv = set(df_existente['fecha'].dt.date)
fechas_faltantes = [f for f in fechas_completas if f.date() not in fechas_en_csv]

print(f"🔍 Se detectaron {len(fechas_faltantes)} fechas faltantes")

# ✅ 4. Definir scraping
base_url = "https://x-y.es/aemet/est-0244X-vilassar-de-dalt?fecha={}"

def extraer_datos_dia(url, fecha_str):
    try:
        r = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        tabla = soup.select_one('div.div_table_scroll table')
        if not tabla:
            print(f"⚠️ Sin tabla para: {fecha_str}")
            return None

        cabecera = tabla.find("tr")
        ths = [th.text.strip().upper() for th in cabecera.find_all("th")]

        # Buscar índices según encabezados
        idx_temp = next((i for i, texto in enumerate(ths) if "TEMP" in texto), None)
        idx_prec = next((i for i, texto in enumerate(ths) if "PRECIPITACIÓN" in texto or "PREC" in texto), None)

        if idx_temp is None or idx_prec is None:
            print(f"⚠️ No se encontraron columnas adecuadas en {fecha_str}")
            return None

        # Filas de datos
        filas = tabla.find_all("tr")[1:]
        temps, precs = [], []

        for fila in filas:
            columnas = fila.find_all("td")
            if len(columnas) > max(idx_temp, idx_prec):
                try:
                    temp = float(columnas[idx_temp].text.replace(",", "."))
                    prec = float(columnas[idx_prec].text.replace(",", "."))
                    temps.append(temp)
                    precs.append(prec)
                except ValueError:
                    continue

        if temps:
            return {
                "fecha": fecha_str,
                "temp_media": round(sum(temps) / len(temps), 2),
                "precipitacion": round(sum(precs), 2)
            }
        else:
            return None

    except Exception as e:
        print(f"❌ Error en {fecha_str}: {e}")
        return None

# ✅ 5. Ejecutar scraping para fechas faltantes
datos_recuperados = []


for fecha in fechas_faltantes[:60]:

    fecha_str = fecha.strftime("%Y-%m-%d")
    url = base_url.format(fecha_str)
    resultado = extraer_datos_dia(url, fecha_str)
    if resultado:
        datos_recuperados.append(resultado)
    time.sleep(0.5)  # evitar bloqueo

# ✅ 6. Combinar con datos existentes
df_nuevos = pd.DataFrame(datos_recuperados)
df_nuevos['fecha'] = pd.to_datetime(df_nuevos['fecha'])

df_final = pd.concat([df_existente, df_nuevos])
df_final = df_final.drop_duplicates(subset="fecha").sort_values("fecha")
df_final.to_csv("clima_vilassar_completo_actualizado.csv", index=False)

print(f"\n📦 CSV actualizado guardado con {len(df_final)} registros como 'clima_vilassar_completo_actualizado.csv'")
df_final.tail()

Se detectó la ausencia de datos para el día 03/05/2023, por lo que se decidió interpolar su valor utilizando los datos de los días inmediatamente anteriores y posteriores.

In [ ]:
import pandas as pd

# Cargar el archivo CSV (ajusta el nombre si es diferente)
df = pd.read_csv("clima_vilassar_completo_actualizado.csv", parse_dates=["fecha"])
import pandas as pd
from datetime import datetime

# Cargar el archivo CSV (ajusta el nombre si es diferente)
df = pd.read_csv("clima_vilassar_completo_actualizado.csv", parse_dates=["fecha"])
df = df.sort_values("fecha")
df.head()
df[(df['fecha'] < datetime(2023, 5, 5)) & (df['fecha'] > datetime(2023, 5, 1))]

In [ ]:
temp_02=df.loc[df['fecha'] == datetime(2023, 5, 2)].iloc[0,1]
temp_04=df.loc[df['fecha'] == datetime(2023, 5, 4)].iloc[0,1]
temp_03=(temp_02+temp_04)/2
temp_03

if not (df['fecha'] == datetime(2023, 5, 3)).any():
    nueva_fila = pd.DataFrame([{
        'fecha': datetime(2023, 5, 3),
        'temp_media': temp_03,
        'precipitacion': 0
    }])
    df= pd.concat([df, nueva_fila], ignore_index=True)
    df = df.sort_values('fecha').reset_index(drop=True)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))
plt.plot(df["fecha"], df["temp_media"], color='tomato', linewidth=1)
plt.title("Temperatura media diaria (Vilassar de Dalt)", fontsize=14)
plt.xlabel("Fecha")
plt.ylabel("Temperatura (°C)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df[(df['fecha'] < datetime(2023, 7, 1)) & (df['fecha'] > datetime(2023, 5, 30))]

Se observa que entre los días 01/06/2023 y 22/06/2023 los datos registrados son erróneos, ya que la página web no muestra valores para esas fechas. Por ello, se optó por obtener los datos correspondientes de la estación meteorológica más cercana.



In [ ]:
fecha_inicio = datetime(2023, 6, 1)
fecha_fin = datetime(2023, 6, 22)
rango_total = pd.date_range(start=fecha_inicio, end=fecha_fin)

base_url = 'https://x-y.es/aemet/est-0201D-barcelona-cmt?fecha={}'
datos_barcelona = []

for fecha in rango_total:
    fecha_str = fecha.strftime("%Y-%m-%d")
    url = base_url.format(fecha_str)  # aquí se construye correctamente
    resultado = extraer_datos_dia(url, fecha_str)  # y ahora sí se usa la URL con la fecha
    if resultado:
        datos_barcelona.append(resultado)
    time.sleep(0.5)


# ✅ 6. Combinar con datos existentes
df_barcelona = pd.DataFrame(datos_recuperados)
df_barcelona['fecha'] = pd.to_datetime(df_nuevos['fecha'])
df_barcelona


In [ ]:
# 2. Eliminar del original las fechas que estén en el nuevo
df_combinado = df[~df['fecha'].isin(df_barcelona['fecha'])]

# 3. Unir los dos dataframes (el original sin duplicados y los nuevos datos)
df_combinado = pd.concat([df_combinado, df_barcelona])

# 4. Ordenar por fecha (opcional pero recomendable)
df_combinado = df_combinado.sort_values('fecha').reset_index(drop=True)
df_combinado.to_csv("clima_completo.csv", index=False)

# Descargar
from google.colab import files
files.download("clima_completo.csv")



In [ ]:
import matplotlib.pyplot as plt
df = pd.read_csv("clima_completo.csv", parse_dates=["fecha"])
plt.figure(figsize=(15, 5))
plt.plot(df_combinado["fecha"], df_combinado["temp_media"], color='tomato', linewidth=1)
plt.title("Temperatura media diaria (Vilassar de Dalt)", fontsize=14)
plt.xlabel("Fecha")
plt.ylabel("Temperatura (°C)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(df["fecha"], df["precipitacion"], color='royalblue', width=1)
plt.title("Precipitación diaria (Vilassar de Dalt)", fontsize=14)
plt.xlabel("Fecha")
plt.ylabel("Precipitación (mm)")
plt.grid(True)
plt.tight_layout()
plt.show()